In [38]:
import itertools

import numpy
import pandas
from numpy import array
from scipy import stats

# 1

Гипотеза о равенстве средних в двух независимых выборках.

# 2

H0: FX1(x) = FX2(x) против H1: FX1(x) = FX2(x+Δ), Δ <≠> 0

# 3

* Перестановочный критерий со статистикой T=1n∑i=1nX1i−1n∑i=1nX2i и Cn2n вариантами расстановки объектов по выборкам.
* Критерий Стьюдента для двух независимых выборок.
* Критерий Манна-Уитни.

# 4

In [10]:
x = array([49, 58, 75, 110, 112, 132, 151, 276, 281, 362])
stats.wilcoxon(x - 200).pvalue

0.28450269791120752

# 5

In [12]:
x1 = array([22, 22, 15, 13, 19, 19, 18, 20, 21, 13, 13, 15])
x2 = array([17, 18, 18, 15, 12, 4, 14, 15, 10])
stats.mannwhitneyu(x2, x1, alternative='less').pvalue

0.029004992720873729

# 6

In [44]:
challenger = pandas.read_table('challenger.txt', sep='\t', header=0)
non_incident = challenger[challenger['Incident'] == 0]['Temperature'].values
incident = challenger[challenger['Incident'] == 1]['Temperature'].values

def get_bootstrap_samples(data, n_samples):
    """
    Copy-pasted from Week 1.
    """
    return data[numpy.random.randint(0, len(data), (n_samples, len(data)))]

def stat_intervals(stat, alpha):
    """
    Copy-pasted from Week 1.
    """
    return numpy.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])

numpy.random.seed(0)
non_incident_samples = numpy.mean(get_bootstrap_samples(non_incident, 1000), axis=1)
incident_samples = numpy.mean(get_bootstrap_samples(incident, 1000), axis=1)
stat_intervals(non_incident_samples - incident_samples, 0.05)

array([ 1.42299107,  7.93861607])

# 7

In [40]:
# Kindly copy-pasted from Emili's notebook.

def permutation_t_stat_ind(sample1, sample2):
    return numpy.mean(sample1) - numpy.mean(sample2)


def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        numpy.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = numpy.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

numpy.random.seed(0)
permutation_test(incident, non_incident, max_permutations = 10000)

0.0057